# Final Assignment "Battle of Neighbourhoods" - Data Science Specialisation

## Part 1: Business Problem

### Geographic location context: Paris

Brussels, capital of France, is home to more than 2,14 million people and the famous Eiffel tower.

### Business Problem: Opening a hotel in Paris
This report aims to solve the problem regarding opening a new hotel in Paris. In which arrondissement ('district') should we open one according to the surrounding and the competition?
Representing a huge investment for an entrereneur or a company, we will base our decision on the results provided by this data science assignment.

### Interested Audience: Investors/Entrepreneurs
This report will be of interest of the individual/organsation which needs the data insights to help taking their decisions to open a hotel in Paris

## Part 2: Data

#### 2.a importing the required libraries

In [1]:
# import required libraries

import numpy as np # lib to handle data in vectorized manner
import pandas as pd #lib for data analyse
import csv
import lxml
import time# import time

# lib to tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

#lib to handle JSON files
import json
from pprint import pprint

# lib to handle request
import requests
#from urllib.request import urlopen as uReq

from pandas import DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# lib to find coordinates
!pip -q install geocoder
import geocoder

# lib convert an address into latitude and longitude values
#!conda install -c conda-forge geopy --yes # uncomment this line if needed
from geopy.geocoders import Nominatim
! pip install geopy

# lib to scrap data from wikipedia
! pip install beautifulsoup4
from bs4 import BeautifulSoup as bsoup

# map rendering lib
#!conda install -c conda-forge folium=0.5.0 --yes  # uncomment this line if needed
! pip install folium
from folium import plugins
import folium

# Matplotlib and associated plotting module
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# lib for displaying images
from IPython.display import Image
from IPython.core.display import HTML

# lib to identify the optimising k for k clusters and import k-means from clustering stage
! pip install yellowbrick
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from yellowbrick.cluster import KElbowVisualizer

print ("libraries downloaded!")

libraries downloaded!


#### 2.b. Data preparation: import data on Paris

I'm using Wikipedia's page on the Districts (Arrondissements) of Paris to scrap the table to create a dataframe. <br>
In order to scrap the data, we will need to library "BeautifulSoup4" imported earlier and "Requests" to create a dataframe with the information "Arrondissement", "Size", "Population". 

FYI - in Paris, the district are named according to their postcode, therefore we will add a new coloumn with the postcode.

#### *IMPORTANT*:
The data retrieved on Wikipedia combined the 1st an 4th district and weren't split in the original document, thus they have been combined under the same postcode "75002" situated in the middle of these tiny district acting as one together.



In [2]:
# get the data on Wikipedia's webpage and store it in the variable url
url = requests.get('https://en.wikipedia.org/wiki/Arrondissements_of_Paris#Arrondissements').text

#scrap the data
soup = bsoup(url, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Arrondissements of Paris - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"9d7cbe3f-9224-46f2-bd79-91f7000cc5b8","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Arrondissements_of_Paris","wgTitle":"Arrondissements of Paris","wgCurRevisionId":969584911,"wgRevisionId":969584911,"wgArticleId":1175882,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 French-language sources (fr)","Articles with short description","Articles with hAudio microformats","CS1 maint: re

In [3]:
response_obj = requests.get('https://en.wikipedia.org/wiki/Arrondissements_of_Paris').text
soup = bsoup(response_obj,'lxml')
Arrondissements_Paris = soup.find('table',{'class':'wikitable'})
Arrondissements_Paris

<table class="wikitable sortable">
<tbody><tr>
<th data-sort-type="number">Arrondissement <br/>(R for <a href="/wiki/Rive_Droite" title="Rive Droite">Right Bank</a>, L for <a href="/wiki/Rive_Gauche" title="Rive Gauche">Left Bank</a>)
</th>
<th>Name
</th>
<th>Area (km<sup>2</sup>)
</th>
<th>Population<br/>(2017 estimate)
</th>
<th>Density (2017)<br/>(inhabitants per km<sup>2</sup>)
</th>
<th>Peak of population
</th>
<th>Mayor
</th>
<th>2020-2026
</th></tr>
<tr align="center">
<td>Paris Centre <a href="/wiki/1st_arrondissement_of_Paris" title="1st arrondissement of Paris">1st (I<sup>er</sup>)</a> / <a href="/wiki/2nd_arrondissement_of_Paris" title="2nd arrondissement of Paris">2nd (II<sup>e</sup>)</a> / <a href="/wiki/3rd_arrondissement_of_Paris" title="3rd arrondissement of Paris">3rd (III<sup>e</sup>)</a>  / <a href="/wiki/4th_arrondissement_of_Paris" title="4th arrondissement of Paris">4th (IV<sup>e</sup>)</a> R</td>
<td>Louvre, Bourse, Temple, Hôtel-de-Ville</td>
<td>5.59 km<sup>2</

In [4]:
totals=Arrondissements_Paris.find_all('tr') # ought to be 18 disticts as the 1st,2nd, 3rd and 4th are combined due to their small size and how the data on Wikipedia is presented
nrows=len(totals)
nrows

18

The Header of the DataFrame is the first row of data

In [5]:
header=totals[0].text.split()
header

['Arrondissement',
 '(R',
 'for',
 'Right',
 'Bank,',
 'L',
 'for',
 'Left',
 'Bank)',
 'Name',
 'Area',
 '(km2)',
 'Population(2017',
 'estimate)',
 'Density',
 '(2017)(inhabitants',
 'per',
 'km2)',
 'Peak',
 'of',
 'population',
 'Mayor',
 '2020-2026']

Checking row 3 as a test

In [6]:
totals[3].text

'\n6th (VIe) L\nLuxembourg\n2.154\xa0km2 (0.832\xa0sq\xa0mi)\n41,976\n19,524\n1911\nJean-Pierre Lecoq (LR)\n\n'

In [7]:
totals[3].text.split('\n')

['',
 '6th (VIe) L',
 'Luxembourg',
 '2.154\xa0km2 (0.832\xa0sq\xa0mi)',
 '41,976',
 '19,524',
 '1911',
 'Jean-Pierre Lecoq (LR)',
 '',
 '']

In [8]:
Arrondissement=totals[3].text.split('\n')[1]
Arrondissement

'6th (VIe) L'

Extracting cells into a dataframe

In [9]:
tables =[]
n=1
while n < nrows :
    Arrondissement=totals[n].text.split('\n')[1]
    Size=totals[n].text.split('\n')[3]
    Population=totals[n].text.split('\n')[5]
    tables.append((Arrondissement,Size,Population))
    n=n+1

df0=pd.DataFrame(tables, columns=['Arrondissement', 'Size km²', 'Population'])
df0.head(17)

,Arrondissement,Size km²,Population
0,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,5.59 km2 (2.16 sq mi),"17,924"
1,5th (Ve) L,2.541 km2 (0.981 sq mi),"23,477"
2,6th (VIe) L,2.154 km2 (0.832 sq mi),"19,524"
3,7th (VIIe) L,4.088 km2 (1.578 sq mi),"12,761"
4,8th (VIIIe) R,3.881 km2 (1.498 sq mi),"9,631"
5,9th (IXe) R,2.179 km2 (0.841 sq mi),"27,556"
6,10th (Xe) R,2.892 km2 (1.117 sq mi),"31,431"
7,11th (XIe) R,3.666 km2 (1.415 sq mi),"40,183"
8,12th (XIIe) R,16.324 km2 (6.303 sq mi)¹6.377 km2 (2.462 sq mi)²,"8,657¹21,729²"
9,13th (XIIIe) L,7.146 km2 (2.759 sq mi),"25,650"


In [10]:
# Add a column with just the postal code numbers

postal_codes = ['75002', '75005', '75006', '75007', '75008', '75009', '75010', '75011', '75012', '75013', '75014', '75015', '75016', '75017', '75018', '75019', '75020']
Arr = pd.DataFrame({'Postcode': postal_codes})
df1 = pd.concat([df0, Arr], axis = 1)
df1.head(17)

,Arrondissement,Size km²,Population,Postcode
0,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,5.59 km2 (2.16 sq mi),"17,924",75002
1,5th (Ve) L,2.541 km2 (0.981 sq mi),"23,477",75005
2,6th (VIe) L,2.154 km2 (0.832 sq mi),"19,524",75006
3,7th (VIIe) L,4.088 km2 (1.578 sq mi),"12,761",75007
4,8th (VIIIe) R,3.881 km2 (1.498 sq mi),"9,631",75008
5,9th (IXe) R,2.179 km2 (0.841 sq mi),"27,556",75009
6,10th (Xe) R,2.892 km2 (1.117 sq mi),"31,431",75010
7,11th (XIe) R,3.666 km2 (1.415 sq mi),"40,183",75011
8,12th (XIIe) R,16.324 km2 (6.303 sq mi)¹6.377 km2 (2.462 sq mi)²,"8,657¹21,729²",75012
9,13th (XIIIe) L,7.146 km2 (2.759 sq mi),"25,650",75013


In [11]:
df1.dtypes

Arrondissement    object
Size km²          object
Population        object
Postcode          object
dtype: object

#### Calculating Latitude and Longitude per Arrondissement

In [12]:
def get_latlng(arcgis_geocoder): #def the fuction 
    
    lat_lng_coords = None  # Initialize the Location
    
    # While loop to geocode all the location coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Paris, France'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [13]:
#testing on an example - the 5th arrondissement which should be 48.85° N, 2.35° E
test = get_latlng('75005')
test

[48.84535000000005, 2.35189172500003]

And reverse geocoding this, using the geocodefarm geocoder, gives the following:

In [14]:
gg = geocoder.geocodefarm(test, method = 'reverse')
gg

<[OK] Geocodefarm - Reverse [46 Rue Monge, 75005 Paris, France]>

Ok the geocoder and function works!

In [15]:
#Get coordinates for all postcodes in df       
start = time.time()
postal_codes = df1['Postcode']    
coordinates = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]
end = time.time()
print("Time of execution: ", end - start, "seconds")

Time of execution:  11.1438307762146 seconds


In [16]:
# Now we combine the coordinates we obtained with the df
df_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df1['Latitude'] = df_coordinates['Latitude']
df1['Longitude'] = df_coordinates['Longitude']
df1.head(17)

,Arrondissement,Size km²,Population,Postcode,Latitude,Longitude
0,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,5.59 km2 (2.16 sq mi),"17,924",75002,48.867715,2.343093
1,5th (Ve) L,2.541 km2 (0.981 sq mi),"23,477",75005,48.845350,2.351892
2,6th (VIe) L,2.154 km2 (0.832 sq mi),"19,524",75006,48.849265,2.332099
3,7th (VIIe) L,4.088 km2 (1.578 sq mi),"12,761",75007,48.854620,2.313061
4,8th (VIIIe) R,3.881 km2 (1.498 sq mi),"9,631",75008,48.872935,2.311249
5,9th (IXe) R,2.179 km2 (0.841 sq mi),"27,556",75009,48.876995,2.337893
6,10th (Xe) R,2.892 km2 (1.117 sq mi),"31,431",75010,48.876155,2.362330
7,11th (XIe) R,3.666 km2 (1.415 sq mi),"40,183",75011,48.859775,2.378126
8,12th (XIIe) R,16.324 km2 (6.303 sq mi)¹6.377 km2 (2.462 sq mi)²,"8,657¹21,729²",75012,48.835120,2.444957
9,13th (XIIIe) L,7.146 km2 (2.759 sq mi),"25,650",75013,48.830090,2.362283


In [17]:
df1.shape


(17, 6)

So there are 17 rows after merging the dataframe with the cvs, with 6 columns after adding Latitude and Longitude

#### Clustering Paris' arrondissements

In [18]:
#mapping Paris
latitude = 48.8566
longitude= 2.3522

In [19]:
# create map of Paris
map_paris = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(df1['Latitude'], df1['Longitude'], df1['Postcode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        parse_html=False).add_to(map_paris)  

map_paris

#### Foursquare API to explore and segment

My Foursquare Credentials

In [20]:
# Foursquare API creditentials
CLIENT_ID = 'HEZLAWN4GXHKE2YLMXWMQTRNKXBBY0R4FMQOOZQLEVIHIQMT' # your Foursquare ID
CLIENT_SECRET = '31ETZTQIRS0PHXHZQ0VJ1TCDOB1300IFFGSJKWDATCM2LD0M' # your Foursquare Secret
VERSION = '20180604'

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HEZLAWN4GXHKE2YLMXWMQTRNKXBBY0R4FMQOOZQLEVIHIQMT
CLIENT_SECRET:31ETZTQIRS0PHXHZQ0VJ1TCDOB1300IFFGSJKWDATCM2LD0M


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Arrondissement', 
                  'AR Latitude', 
                  'AR Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category' ]
    
    return(nearby_venues)

#### Writting the code to run the function on each arrondissement and create a new dataframe (Paris_venues)

In [22]:
Paris_venues = getNearbyVenues(names=df1['Arrondissement'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                    )

Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe)  / 4th (IVe) R
5th (Ve) L
6th (VIe) L
7th (VIIe) L
8th (VIIIe) R
9th (IXe) R
10th (Xe) R
11th (XIe) R
12th (XIIe) R
13th (XIIIe) L
14th (XIVe) L
15th (XVe) L
16th (XVIe) R
17th (XVIIe) R
18th (XVIIIe) R
19th (XIXe) R
20th (XXe) R


In [23]:
print(Paris_venues.shape)
Paris_venues.head(15)

(942, 7)


,Arrondissement,AR Latitude,AR Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,48.867715,2.343093,Pizzeria Popolare,48.868074,2.343379,Pizza Place
1,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,48.867715,2.343093,Le Moderne,48.868856,2.342142,French Restaurant
2,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,48.867715,2.343093,Le Nom M'échappe,48.868125,2.341673,Bistro
3,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,48.867715,2.343093,Librairie Gourmande,48.866951,2.343996,Bookstore
4,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,48.867715,2.343093,Nose,48.866242,2.344885,Perfume Shop
5,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,48.867715,2.343093,Devialet,48.868108,2.343808,Music Store
6,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,48.867715,2.343093,Matamata,48.866044,2.343956,Coffee Shop
7,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,48.867715,2.343093,Place des Victoires,48.865759,2.341219,Plaza
8,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,48.867715,2.343093,Le Moulin de la Vierge,48.866202,2.341155,Bakery
9,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,48.867715,2.343093,Galerie Vivienne,48.866731,2.339800,Historic Site


In [24]:
#Number of venues per Arrondissement
Paris_venues.groupby('Arrondissement').count()

,AR Latitude,AR Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Arrondissement,,,,,,
10th (Xe) R,100,100,100,100,100,100
11th (XIe) R,46,46,46,46,46,46
12th (XIIe) R,13,13,13,13,13,13
13th (XIIIe) L,30,30,30,30,30,30
14th (XIVe) L,26,26,26,26,26,26
15th (XVe) L,64,64,64,64,64,64
16th (XVIe) R,4,4,4,4,4,4
17th (XVIIe) R,57,57,57,57,57,57
18th (XVIIIe) R,48,48,48,48,48,48


We can see that the 16th, 12th arrondissement don't have much venues and are pretty isolated. Whereas, the 1st-4th, 9th, 5th, and 10th are the more popular.

In [25]:
#Number of unique categories
print('The number of unique categories is {}.'.format(len(Paris_venues['Venue Category'].unique())))

The number of unique categories is 182.


#### Analyzing arrondissement

In [26]:
# one hot encoding
Paris_onehot = pd.get_dummies(Paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Paris_onehot['Arrondissement'] = Paris_venues['Arrondissement'] 

# move district column to the first column
cols=list(Paris_onehot.columns.values)
cols.pop(cols.index('Arrondissement'))
Paris_onehot=Paris_onehot[['Arrondissement']+cols]

# rename Neighborhood for Districts so that future merge works
Paris_onehot.rename(columns = {'Arrondissement': 'Arrondissement'}, inplace = True)
Paris_onehot.head(17)

,Arrondissement,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Botanical Garden,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Line,Bus Stop,Café,Cambodian Restaurant,Canal,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Noodle House,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Snack Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,

In [27]:
Paris_onehot.shape

(942, 183)

#### Grouping rows by arrondissement and by the frequency mean of occurrence of each category

In [28]:
Paris_grouped = Paris_onehot.groupby('Arrondissement').mean().reset_index()
Paris_grouped

,Arrondissement,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Botanical Garden,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Line,Bus Stop,Café,Cambodian Restaurant,Canal,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Noodle House,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Snack Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,10th (Xe) R,0.000000,0.020000,0.0000,0.00,0.00,0.00000,0.000000,0.000000,0.040000,0.0000,0.020000,0.000000,0.030000,0.000000,0.00,0.000000,0.010000,0.000000,0.00,0.000000,0.040000,0.010000,0.000000,0.01,0.000000,0.00,0.02,0.00,0.020000,0.000000,0.000000,0.050000,0.000000,0.01,0.000000,0.010000,0.000000,0.00,0.000000,0.010000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.010000,0.000000,0.000000,0.000000,0.010000,0.010000,0.01,0.00,0.0000,0.000000,0.00,0.00,0.010000,0.00,0.000000,0.000000,0.01,0.010000,0.0000,0.110000,0.010000,0.010000,0.000000,0.00,0.010000,0.0000,0.0000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.000000,0.00,0.010000,0.020000,0.000000,0.010000,0.020000,0.000000,0.00,0.000000,0.01,0.020000,0.020000,0.000000,0.010000,0.0000,0.020000,0.00,0.000000,0.010000,0.00,0.000000,0.0000,0.020000,0.000000,0.000000,0.010000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.01,0.00,0.0000,0.00,0.0000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.010000,0.010000,0.00,0.000000,0.00,0.010000,0.01,0.00,0.0100,0.00,0.00,0.000000,0.01,0.0000,0.000000,0.01,0.000000,0.0000,0.000000,0.000000,0.00,0.01,0.0000,0.00,0.00,0.000000,0.0000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000

In [29]:
Paris_grouped.shape

(17, 183)

#### Printing the arrondissement and the top 5 most common venues

In [30]:
num_top_venues = 5

for hood in Paris_grouped['Arrondissement']:
    print("----"+hood+"----")
    temp = Paris_grouped[Paris_grouped['Arrondissement'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----10th (Xe) R----
               venue  freq
0  French Restaurant  0.11
1               Café  0.05
2        Coffee Shop  0.05
3   Asian Restaurant  0.04
4             Bistro  0.04


----11th (XIe) R----
                           venue  freq
0              French Restaurant  0.11
1                       Wine Bar  0.07
2                     Restaurant  0.07
3                           Café  0.07
4  Vegetarian / Vegan Restaurant  0.04


----12th (XIIe) R----
               venue  freq
0            Theater  0.23
1     Tennis Stadium  0.08
2        Comedy Club  0.08
3  Recreation Center  0.08
4   Botanical Garden  0.08


----13th (XIIIe) L----
                   venue  freq
0  Vietnamese Restaurant  0.20
1      French Restaurant  0.10
2       Asian Restaurant  0.10
3     Chinese Restaurant  0.10
4                  Hotel  0.07


----14th (XIVe) L----
                        venue  freq
0           French Restaurant  0.42
1                      Bakery  0.08
2                       Hotel  0

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Arrondissement']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
arrondissement_venues_sorted = pd.DataFrame(columns=columns)
arrondissement_venues_sorted['Arrondissement'] = Paris_grouped['Arrondissement']

for ind in np.arange(Paris_grouped.shape[0]):
    arrondissement_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Paris_grouped.iloc[ind, :], num_top_venues)

arrondissement_venues_sorted

,Arrondissement,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,10th (Xe) R,French Restaurant,Café,Coffee Shop,Bistro,Asian Restaurant,Pizza Place,Bar,Thai Restaurant,Wine Bar,Burger Joint
1,11th (XIe) R,French Restaurant,Café,Wine Bar,Restaurant,Pastry Shop,Italian Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bar,Asian Restaurant
2,12th (XIIe) R,Theater,Comedy Club,Recreation Center,Restaurant,Performing Arts Venue,Fast Food Restaurant,Botanical Garden,Stadium,Bike Trail,Tennis Stadium
3,13th (XIIIe) L,Vietnamese Restaurant,French Restaurant,Chinese Restaurant,Asian Restaurant,Hotel,Thai Restaurant,Sushi Restaurant,Convenience Store,Creperie,Dance Studio
4,14th (XIVe) L,French Restaurant,Hotel,Bakery,Thai Restaurant,Pizza Place,Fast Food Restaurant,Café,Food & Drink Shop,Modern European Restaurant,Brasserie
5,15th (XVe) L,French Restaurant,Italian Restaurant,Hotel,Coffee Shop,Bistro,Bakery,Indian Restaurant,Supermarket,Lebanese Restaurant,Park
6,16th (XVIe) R,Bike Rental / Bike Share,Park,Lake,Circus,Yoga Studio,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
7,17th (XVIIe) R,French Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Plaza,Café,Restaurant,Bistro,Supermarket
8,18th (XVIIIe) R,Bar,French Restaurant,Gastropub,Italian Restaurant,Convenience Store,Coffee Shop,Restaurant,Café,Bistro,Middle Eastern Restaurant
9,19th (XIXe) R,French Restaurant,Supermarket,Bakery,Hotel,Metro Station,Lebanese Restaurant,Beer Garden,Moroccan Restaurant,Bar,Scenic Lookout


As we can see the most common venue is French Restaurant. We can see as well that hotels are more common in the first the 7th then in the 14th, 17th, 8th, 9th. Which makes sense as these are the touristic and/or wealthy areas in Paris.

It's interesting to notice as well, in case we are aiming to target a wealthy and aging between Gen X and Gen Y, we could aim at the 8th district as the common nearby venues are diversify (French Restaurant for the the typical experience, Art Gallery, Department store (such as Gallery Lafayette), Spa and cocktail bars) - a very convenient location for a hotel! 

#### Clustering Arrondissement
Let's now perform a clusering of the arrondissement <br>
To determine what will be k clusters, we will use the Elbow Method

#### New dataframe for clusters and top 10 venues for each district

In [33]:
Paris_merged = df1

Paris_grouped_clustering = Paris_grouped.drop('Arrondissement', 1)

# set number of clusters
kclusters = 9

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[1:10]

# add clustering labels
Paris_merged['Cluster Labels'] = kmeans.labels_

# merge Paris_grouped with Paris_data to add latitude/longitude for each arrondissement
Paris_merged = Paris_merged.join(arrondissement_venues_sorted.set_index('Arrondissement'), on='Arrondissement')

Paris_merged.head(11) # check the last columns!





,Arrondissement,Size km²,Population,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,5.59 km2 (2.16 sq mi),"17,924",75002,48.867715,2.343093,6,French Restaurant,Cocktail Bar,Bakery,Bistro,Italian Restaurant,Wine Bar,Japanese Restaurant,Salad Place,Coffee Shop,Thai Restaurant
1,5th (Ve) L,2.541 km2 (0.981 sq mi),"23,477",75005,48.845350,2.351892,6,French Restaurant,Bar,Hotel,Wine Bar,Italian Restaurant,Creperie,Café,Coffee Shop,Pub,Science Museum
2,6th (VIe) L,2.154 km2 (0.832 sq mi),"19,524",75006,48.849265,2.332099,0,French Restaurant,Italian Restaurant,Hotel,Restaurant,Plaza,Chocolate Shop,Fountain,Pastry Shop,Bakery,Tea Room
3,7th (VIIe) L,4.088 km2 (1.578 sq mi),"12,761",75007,48.854620,2.313061,1,Hotel,French Restaurant,Café,Italian Restaurant,History Museum,Plaza,Garden,Historic Site,Coffee Shop,Japanese Restaurant
4,8th (VIIIe) R,3.881 km2 (1.498 sq mi),"9,631",75008,48.872935,2.311249,3,French Restaurant,Hotel,Art Gallery,Department Store,Spa,Cocktail Bar,Cosmetics Shop,Bakery,Salad Place,Sporting Goods Shop
5,9th (IXe) R,2.179 km2 (0.841 sq mi),"27,556",75009,48.876995,2.337893,6,French Restaurant,Hotel,Japanese Restaurant,Bistro,Bakery,Wine Bar,Cocktail Bar,Lounge,Bar,Gym / Fitness Center
6,10th (Xe) R,2.892 km2 (1.117 sq mi),"31,431",75010,48.876155,2.362330,4,French Restaurant,Café,Coffee Shop,Bistro,Asian Restaurant,Pizza Place,Bar,Thai Restaurant,Wine Bar,Burger Joint
7,11th (XIe) R,3.666 km2 (1.415 sq mi),"40,183",75011,48.859775,2.378126,5,French Restaurant,Café,Wine Bar,Restaurant,Pastry Shop,Italian Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bar,Asian Restaurant
8,12th (XIIe) R,16.324 km2 (6.303 sq mi)¹6.377 km2 (2.462 sq mi)²,"8,657¹21,729²",75012,48.835120,2.444957,8,Theater,Comedy Club,Recreation Center,Restaurant,Performing Arts Venue,Fast Food Restaurant,Botanical Garden,Stadium,Bike Trail,Tennis Stadium
9,13th (XIIIe) L,7.146 km2 (2.759 sq mi),"25,650",75013,48.830090,2.362283,7,Vietnamese Restaurant,French Restaurant,Chinese Restaurant,Asian Restaurant,Hotel,Thai Restaurant,Sushi Restaurant,Convenience Store,Creperie,Dance Studio


## Part 4: Results

#### Clusters visualization of Paris's arrondissement

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Paris_merged['Latitude'], Paris_merged['Longitude'], Paris_merged['Arrondissement'], Paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 0 - Most common venues: Restaurants and bars

In [35]:
# Cluster 0
Paris_merged.loc[Paris_merged['Cluster Labels'] == 0, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]

,Arrondissement,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,6th (VIe) L,2.332099,0,French Restaurant,Italian Restaurant,Hotel,Restaurant,Plaza,Chocolate Shop,Fountain,Pastry Shop,Bakery,Tea Room


#### Cluster 1 - Most common venues: Restaurants, Hotels and Bars

In [36]:
# Cluster 1
Paris_merged.loc[Paris_merged['Cluster Labels'] == 1, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]

,Arrondissement,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,7th (VIIe) L,2.313061,1,Hotel,French Restaurant,Café,Italian Restaurant,History Museum,Plaza,Garden,Historic Site,Coffee Shop,Japanese Restaurant


#### Cluster 2 - Most common venues: Restaurants and parks (No Hotels in Top 10 venues)

In [37]:
# Cluster 2
Paris_merged.loc[Paris_merged['Cluster Labels'] == 2, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]

,Arrondissement,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,14th (XIVe) L,2.323828,2,French Restaurant,Hotel,Bakery,Thai Restaurant,Pizza Place,Fast Food Restaurant,Café,Food & Drink Shop,Modern European Restaurant,Brasserie
11,15th (XVe) L,2.296975,2,French Restaurant,Italian Restaurant,Hotel,Coffee Shop,Bistro,Bakery,Indian Restaurant,Supermarket,Lebanese Restaurant,Park
15,19th (XIXe) R,2.385625,2,French Restaurant,Supermarket,Bakery,Hotel,Metro Station,Lebanese Restaurant,Beer Garden,Moroccan Restaurant,Bar,Scenic Lookout



#### Cluster 3 - Most common venues: Nightclubs , Stations, Bars (No Hotels in Top 10 venues

In [38]:
# Cluster 3
Paris_merged.loc[Paris_merged['Cluster Labels'] == 3, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]

,Arrondissement,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,8th (VIIIe) R,2.311249,3,French Restaurant,Hotel,Art Gallery,Department Store,Spa,Cocktail Bar,Cosmetics Shop,Bakery,Salad Place,Sporting Goods Shop


In [39]:
# Cluster 4
Paris_merged.loc[Paris_merged['Cluster Labels'] == 4, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]

,Arrondissement,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,10th (Xe) R,2.36233,4,French Restaurant,Café,Coffee Shop,Bistro,Asian Restaurant,Pizza Place,Bar,Thai Restaurant,Wine Bar,Burger Joint


In [40]:
# Cluster 5
Paris_merged.loc[Paris_merged['Cluster Labels'] == 5, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]

,Arrondissement,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,11th (XIe) R,2.378126,5,French Restaurant,Café,Wine Bar,Restaurant,Pastry Shop,Italian Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bar,Asian Restaurant
13,17th (XVIIe) R,2.306293,5,French Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Plaza,Café,Restaurant,Bistro,Supermarket
14,18th (XVIIIe) R,2.348094,5,Bar,French Restaurant,Gastropub,Italian Restaurant,Convenience Store,Coffee Shop,Restaurant,Café,Bistro,Middle Eastern Restaurant


In [41]:
# Cluster 6
Paris_merged.loc[Paris_merged['Cluster Labels'] == 6, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]

,Arrondissement,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,2.343093,6,French Restaurant,Cocktail Bar,Bakery,Bistro,Italian Restaurant,Wine Bar,Japanese Restaurant,Salad Place,Coffee Shop,Thai Restaurant
1,5th (Ve) L,2.351892,6,French Restaurant,Bar,Hotel,Wine Bar,Italian Restaurant,Creperie,Café,Coffee Shop,Pub,Science Museum
5,9th (IXe) R,2.337893,6,French Restaurant,Hotel,Japanese Restaurant,Bistro,Bakery,Wine Bar,Cocktail Bar,Lounge,Bar,Gym / Fitness Center
12,16th (XVIe) R,2.255971,6,Bike Rental / Bike Share,Park,Lake,Circus,Yoga Studio,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
16,20th (XXe) R,2.400828,6,French Restaurant,Japanese Restaurant,Bakery,Pizza Place,Café,Bus Stop,Bistro,Sushi Restaurant,Bar,Hotel


In [42]:
# Cluster 7
Paris_merged.loc[Paris_merged['Cluster Labels'] == 7, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]

,Arrondissement,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,13th (XIIIe) L,2.362283,7,Vietnamese Restaurant,French Restaurant,Chinese Restaurant,Asian Restaurant,Hotel,Thai Restaurant,Sushi Restaurant,Convenience Store,Creperie,Dance Studio


In [43]:
# Cluster 8
Paris_merged.loc[Paris_merged['Cluster Labels'] == 8, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]

,Arrondissement,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,12th (XIIe) R,2.444957,8,Theater,Comedy Club,Recreation Center,Restaurant,Performing Arts Venue,Fast Food Restaurant,Botanical Garden,Stadium,Bike Trail,Tennis Stadium


## Part 5: Discussion

The results shows the different district (arrondissement) in Paris and the most common venues per arrondissement and clusters.

We see for example that  the 1- 4th, 5th, 9th, 16th and 20th are very popular for restaurants.

As seen earlier the 8th district is quite interesting if the audience targeted is a wealthy and aging population between Gen X and Gen Y, we could aim at the 8th district as the common nearby venues are diversify (French Restaurant for the the typical experience, Art Gallery, Department store (such as Gallery Lafayette), Spa and cocktail bars) - a very convenient location for a hotel! 


## Part 6: Conclusion

The results are a bit vaste and the original data combining the 1st to 4th arrondissement together as they're a small, isn't helping.
In order to proceed with a decision (confirming the implementation in the 8th district) more accurate data should be gathered. Unfortunately the current data haven't been updated yet.

It would be interesting to add the suburbs sourrounding Paris such as Levallois, Genviliers, Saint-Denis, and Montreuil, getting famous right now with real estate (professional and touristic).

Thank you for reviewing my work! Please don't hesitate to give me some feedback so I can improve my work (but would appreciate if you could avoid failing me as m hands are tight with the deadlines, thanks! :) )